In [1]:
import pandas as pd
import numpy as np
import re

import json

In [2]:
def filterString(string):
    strOut = string.replace('\r',' ').replace('\n',' ').replace('\t',' ')
    return strOut

In [3]:
path = 'data-seed/finance/'
with open('data-seed/seed_tweets.json', 'r') as seedfile:
    seed = json.load(seedfile)

In [6]:
path = 'data-candidates/finance/'
with open('data-candidates/candidate_tweets.json', 'r') as candidatefile:
    candidates = json.load(candidatefile)

In [7]:
tweet_header = 'id_tweet\tid_user\tscreen_name\tlang\tfavourite_count\tcreate_at\tretweet_count\ttext\n'
mention_header = 'id_tweet\tid_user\tscreen_name\n'
hashtag_header = 'id_tweet\ttag\n'
user_header = 'id_user\tscreen_name\n'
with open(path+'user.csv', 'w') as userfile:
    with open(path+'tweet.csv', 'w') as tweetfile:
        with open(path+'mention.csv', 'w') as mentionfile:
            with open(path+'tag.csv', 'w') as tagfile:
                tweetfile.write(tweet_header)
                mentionfile.write(mention_header)
                tagfile.write(hashtag_header)
                userfile.write(user_header)
                for user in candidates:
                    for tweet in candidates[user]:
                        #  tweet data
                        id_tweet = tweet['_id']
                        id_user = tweet['id_user']
                        screen_name = tweet['screen_name'].lower()
                        lang = tweet['lang']
                        fav_count = tweet['favourite_count']
                        timestamp = tweet['create_at']
                        rt_count = tweet['retweet_count']
                        text = filterString(tweet['text']).encode('utf-8')
                        tweetrow = '{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(id_tweet,id_user,screen_name,lang,fav_count,timestamp,rt_count,text)
                        tweetfile.write(tweetrow)
                        
                        # hashtag data
                        p = re.compile("(?<=^|(?<=[^a-zA-Z0-9-_\.]))#([A-Za-z]+[A-Za-z0-9]+)")
                        hashtags = p.findall(text)
                        for tag in hashtags:
                            tagrow = '{}\t{}\n'.format(id_tweet, tag.lower())
                            tagfile.write(tagrow)

                        # mention data
                        mentions = tweet['mentions']
                        for m in mentions:
                            id_user_mentioned = m['id']
                            screen_name_mentioned = m['screen_name'].lower()
                            mentionrow = '{}\t{}\t{}\n'.format(id_tweet, id_user_mentioned, screen_name_mentioned)
                            mentionfile.write(mentionrow)
                        
                    # user data
                    userrow = '{}\t{}\n'.format(id_user,screen_name)
                    userfile.write(userrow)
            

Statistics

In [43]:
pathseed = 'data-seed/finance/'
pathcandidate = 'data-candidates/finance/'

for table in ['user', 'tweet', 'tag', 'mention']:
    seeddata = pd.read_csv(pathseed+table+'.csv', sep='\t', quoting=3).shape[0]
    candidatedata = pd.read_csv(pathcandidate+table+'.csv', sep='\t', quoting=3).shape[0]
    print '{}: {}'.format(table.upper(), seeddata+candidatedata)
    print 'Seeds: {}'.format(seeddata)
    print 'Candidates: {}'.format(candidatedata)
    print '---------------'

USER: 925
Seeds: 10
Candidates: 915
---------------
TWEET: 178907
Seeds: 1970
Candidates: 176937
---------------
TAG: 27653
Seeds: 219
Candidates: 27434
---------------
MENTION: 176391
Seeds: 1689
Candidates: 174702
---------------


In [36]:
tweet = pd.read_csv(pathcandidate+'tweet.csv', sep='\t', quoting=3)
mention = pd.read_csv(pathcandidate+'mention.csv', sep='\t')

In [37]:
tweet.head()

,id_tweet,id_user,screen_name,lang,favourite_count,create_at,retweet_coun,text
0,966850148381978624,31432492,maggielake,en,18,2018-02-23 01:40:13,6,"“It is not about being liberal, Dem or Republi..."
1,959182226800959488,31432492,maggielake,en,0,2018-02-01 21:50:38,5270,RT @rihanna: Merci @emmanuelmacron for steppin...
2,959182200129380355,31432492,maggielake,en,0,2018-02-01 21:50:31,2821,RT @rihanna: hi @JulieBishopMP &amp; @Turnbull...
3,959182152477921285,31432492,maggielake,en,0,2018-02-01 21:50:20,4325,RT @rihanna: 🇬🇧 Hello @theresa_may and @Penn...
4,951144922719162373,31432492,maggielake,en,9,2018-01-10 17:33:15,2,Her dress design changed everything https://t....


In [38]:
mention.head()

,id_tweet,id_user,screen_name
0,959182226800959488,79293791,rihanna
1,959182226800959488,1976143068,emmanuelmacron
2,959182226800959488,171314974,gpforeducation
3,959182200129380355,79293791,rihanna
4,959182200129380355,89856037,juliebishopmp


In [39]:
test = tweet[['id_tweet','text']].merge(mention, on='id_tweet', how='right')

In [40]:
test[test['text'].isnull()]

,id_tweet,text,id_user,screen_name


In [41]:
tweet.shape

(176937, 8)